In [26]:
import yfinance as yf
import numpy as np
from statsmodels.tsa.stattools import adfuller
from scipy.stats import f
import pandas as pd
import itertools
from statsmodels.tsa.arima.model import ARIMA
import warnings
import matplotlib.pyplot as plt
import statsmodels.api as sm
from statsmodels.stats.diagnostic import het_arch
from arch import arch_model
from scipy.stats import chi2
warnings.filterwarnings("ignore")


In [27]:
df = yf.Ticker("^GSPC").history(period="max")
df['daily_log_returns'] = np.log(df['Close'] / df['Close'].shift(1))
df = df.dropna()

log_returns = df['daily_log_returns']


event_windows = [
    ("dot_com_bubble",   "1997-01-01", "2000-03-10", "2003-01-01"),
    ("financial_crisis", "2006-01-01", "2008-09-15", "2011-01-01"),
    ("flash_crash",      "2008-05-06", "2010-05-06", "2012-05-06"),
    ("covid_crash",      "2018-03-11", "2020-03-11", "2022-03-11"),
    ("russia_invasion",  "2020-02-24", "2022-02-24", "2024-02-24"),
]

structured_periods = []
for name, start, event_date, end in event_windows:
    period_data = log_returns.loc[start:end]
    structured_periods.append({
        "event": name,
        "start": start,
        "event_date": event_date,
        "end": end,
        "returns": period_data
    })

for p in structured_periods:
    print(f"{p['event']} → {p['start']} to {p['end']} (event at {p['event_date']})")
    print(f"  Observations: {len(p['returns'])}\n")

dot_com_bubble → 1997-01-01 to 2003-01-01 (event at 2000-03-10)
  Observations: 1509

financial_crisis → 2006-01-01 to 2011-01-01 (event at 2008-09-15)
  Observations: 1259

flash_crash → 2008-05-06 to 2012-05-06 (event at 2010-05-06)
  Observations: 1009

covid_crash → 2018-03-11 to 2022-03-11 (event at 2020-03-11)
  Observations: 1009

russia_invasion → 2020-02-24 to 2024-02-24 (event at 2022-02-24)
  Observations: 1008



## Simple variance test (overall level, not structural) using F-distribution

In [28]:
def f_test_variance(x1, x2):
    s1_sq = np.var(x1, ddof=1)
    s2_sq = np.var(x2, ddof=1)
    F_stat = s1_sq / s2_sq
    df1 = len(x1) - 1
    df2 = len(x2) - 1
    # Two-sided p-value
    p_value = 2 * min(f.cdf(F_stat, df1, df2), 1 - f.cdf(F_stat, df1, df2))
    return s1_sq, s2_sq, F_stat, p_value

f_test_results = []
for name, start, event_date, end in event_windows:
    data_full = log_returns.loc[start:end]

    before = data_full.loc[start:event_date].iloc[:-1]  
    after  = data_full.loc[event_date:]

    s1, s2, F_stat, p_val = f_test_variance(before, after)
    f_test_results.append({
        "event": name,
        "start": start,
        "event_date": event_date,
        "end": end,
        "var_before": s1,
        "var_after": s2,
        "F_stat": F_stat,
        "p_value": p_val
    })

f_test_df = pd.DataFrame(f_test_results)
print("=== F-test variance results ===")
print(f_test_df)
print()

=== F-test variance results ===
              event       start  event_date         end  var_before  \
0    dot_com_bubble  1997-01-01  2000-03-10  2003-01-01    0.000146   
1  financial_crisis  2006-01-01  2008-09-15  2011-01-01    0.000101   
2       flash_crash  2008-05-06  2010-05-06  2012-05-06    0.000464   
3       covid_crash  2018-03-11  2020-03-11  2022-03-11    0.000117   
4   russia_invasion  2020-02-24  2022-02-24  2024-02-24    0.000280   

   var_after    F_stat       p_value  
0   0.000215  0.677315  9.206945e-08  
1   0.000419  0.241116  9.501963e-68  
2   0.000166  2.797905  2.220446e-16  
3   0.000249  0.468542  4.473343e-17  
4   0.000143  1.959486  8.215650e-14  



Large differences for especially financial crisis

## Fitting ARMA models & testing for ARCH effects

Purpose is models for residuals

ARCH effects are tested using ARCH-LM test

In [29]:

p_values = range(4)
q_values = range(4)

arch_lm_results = []
for name, start, event_date, end in event_windows:
    data = log_returns.loc[start:end]

    best_aic = np.inf
    best_order = None
    best_model = None

    for p, q in itertools.product(p_values, q_values):
        try:
            model = ARIMA(data, order=(p, 0, q)).fit()
            if model.aic < best_aic:
                best_aic = model.aic
                best_order = (p, q)
                best_model = model
        except:
            continue

    if best_model is None:
        arch_lm_results.append({
            "event": name,
            "p": None,
            "q": None,
            "LM_stat": None,
            "p_value": None,
            "ARCH_effects": "Model fitting failed"
        })
        continue

    #ARCH-LM test
    residuals = best_model.resid
    lm_test = het_arch(residuals)
    LM_stat = lm_test[0]
    LM_pvalue = lm_test[1]

    arch_lm_results.append({
        "event": name,
        "p": best_order[0],
        "q": best_order[1],
        "LM_stat": LM_stat,
        "p_value": LM_pvalue,
        "ARCH_effects": "Yes" if LM_pvalue < 0.05 else "No"
    })

arch_lm_df = pd.DataFrame(arch_lm_results)
print("=== ARMA best orders & ARCH-LM test ===")
print(arch_lm_df)
print()


=== ARMA best orders & ARCH-LM test ===
              event  p  q     LM_stat       p_value ARCH_effects
0    dot_com_bubble  0  0  106.888422  2.259278e-18          Yes
1  financial_crisis  3  0  398.177955  2.298807e-79          Yes
2       flash_crash  2  0  293.699580  3.335327e-57          Yes
3       covid_crash  3  2  334.970642  6.140149e-66          Yes
4   russia_invasion  0  3  305.627744  1.003831e-59          Yes



## Fitting garch

In [30]:
garch_results = []
for _, row in arch_lm_df.iterrows():
    event = row["event"]
    p_val = row["p"]
    q_val = row["q"]

    # Skip if ARMA order not found
    if pd.isnull(p_val) or pd.isnull(q_val):
        continue

    p_val = int(p_val)
    q_val = int(q_val)

    (__, start, event_date, end) = next(x for x in event_windows if x[0] == event)
    data = log_returns.loc[start:end]

    try:
        arma_model = ARIMA(data, order=(p_val, 0, q_val)).fit()
        residuals = arma_model.resid
    except:
        continue

    best_aic = np.inf
    best_bic = np.inf
    best_aic_model = None
    best_bic_model = None
    best_aic_order = None
    best_bic_order = None

    for gp, gq in itertools.product(range(1, 4), range(1, 4)):
        try:
            garch_mod = arch_model(residuals, vol='Garch', p=gp, q=gq, mean='Zero')
            garch_fit = garch_mod.fit(disp="off")

            if garch_fit.aic < best_aic:
                best_aic = garch_fit.aic
                best_aic_order = (gp, gq)
                best_aic_model = garch_fit

            if garch_fit.bic < best_bic:
                best_bic = garch_fit.bic
                best_bic_order = (gp, gq)
                best_bic_model = garch_fit
        except:
            pass

    garch_results.append({
        "event": event,
        "ARMA(p,q)": (p_val, q_val),
        "GARCH(p,q) AIC": best_aic_order,
        "AIC": best_aic,
        "GARCH(p,q) BIC": best_bic_order,
        "BIC": best_bic,
        "model_AIC": best_aic_model,
        "model_BIC": best_bic_model
    })

garch_summary_df = pd.DataFrame([{
    "event": r["event"],
    "ARMA(p,q)": r["ARMA(p,q)"],
    "Best GARCH(p,q) AIC": r["GARCH(p,q) AIC"],
    "AIC": r["AIC"],
    "Best GARCH(p,q) BIC": r["GARCH(p,q) BIC"],
    "BIC": r["BIC"]
} for r in garch_results])

print("=== GARCH summary results ===")
print(garch_summary_df)
print()


=== GARCH summary results ===
              event ARMA(p,q) Best GARCH(p,q) AIC          AIC  \
0    dot_com_bubble    (0, 0)              (2, 3) -8896.810455   
1  financial_crisis    (3, 0)              (3, 1) -7677.692660   
2       flash_crash    (2, 0)              (3, 3) -5855.022640   
3       covid_crash    (3, 2)              (2, 1) -6475.197944   
4   russia_invasion    (0, 3)              (1, 2) -6199.471489   

  Best GARCH(p,q) BIC          BIC  
0              (1, 1) -8871.086035  
1              (3, 1) -7652.002295  
2              (2, 1) -5828.006177  
3              (1, 1) -6458.179392  
4              (1, 2) -6179.808595  



/Library/Frameworks/Python.framework/Versions/3.13/lib/python3.13/site-packages/arch/univariate/base.py:768: ConvergenceWarning: The optimizer returned code 8. The message is:
Positive directional derivative for linesearch
See scipy.optimize.fmin_slsqp for code meaning.

  warnings.warn(
/Library/Frameworks/Python.framework/Versions/3.13/lib/python3.13/site-packages/arch/univariate/base.py:768: ConvergenceWarning: The optimizer returned code 8. The message is:
Positive directional derivative for linesearch
See scipy.optimize.fmin_slsqp for code meaning.

  warnings.warn(
/Library/Frameworks/Python.framework/Versions/3.13/lib/python3.13/site-packages/arch/univariate/base.py:768: ConvergenceWarning: The optimizer returned code 8. The message is:
Positive directional derivative for linesearch
See scipy.optimize.fmin_slsqp for code meaning.

  warnings.warn(


In [16]:
def likelihood_ratio_test(event_name, full_model, arma_order, garch_order, full_data, break_date):
    try:
        before_data = full_data.loc[:break_date]
        after_data = full_data.loc[break_date:]

        arma_before = ARIMA(before_data, order=(arma_order[0], 0, arma_order[1])).fit()
        arma_after = ARIMA(after_data, order=(arma_order[0], 0, arma_order[1])).fit()

        resid_before = arma_before.resid
        resid_after = arma_after.resid

        garch_before = arch_model(resid_before, vol="Garch", p=garch_order[0], q=garch_order[1], mean="Zero").fit(disp="off")
        garch_after = arch_model(resid_after, vol="Garch", p=garch_order[0], q=garch_order[1], mean="Zero").fit(disp="off")

        ll_restricted = full_model.loglikelihood
        ll_unrestricted = garch_before.loglikelihood + garch_after.loglikelihood
        LR_stat = -2 * (ll_restricted - ll_unrestricted)
        df = 3
        p_value = 1 - chi2.cdf(LR_stat, df)

        return {
            "event": event_name,
            "ARMA(p,q)": arma_order,
            "GARCH(p,q)": garch_order,
            "LR_stat": LR_stat,
            "p_value": p_value,
            "significant": "Yes" if p_value < 0.05 else "No",
            "garch_before": garch_before,
            "garch_after": garch_after
        }

    except Exception as e:
        return {
            "event": event_name,
            "error": str(e)
        }


In [31]:
lr_results = []

for g in garch_results:
    event_name = g["event"]
    arma_order = g["ARMA(p,q)"]
    garch_order = g["GARCH(p,q) AIC"]
    full_model = g["model_AIC"]

    period = next(p for p in structured_periods if p["event"] == event_name)
    full_data = period["returns"]
    break_date = period["event_date"]

    result = likelihood_ratio_test(event_name, full_model, arma_order, garch_order, full_data, break_date)
    lr_results.append(result)

lr_df = pd.DataFrame(lr_results)

for result in lr_results:
    print(f"\n================== {result['event'].upper()} ==================")
    
    if 'error' in result:
        print(f"Error: {result['error']}")
        continue

    print(f"Significant difference in GARCH parameters? → {result['significant']}")
    print(f"LR statistic: {result['LR_stat']:.4f}, p-value: {result['p_value']:.4f}")
    
    print("\n--- GARCH Model (Before Event) ---")
    print(result["garch_before"].summary())
    
    print("\n--- GARCH Model (After Event) ---")
    print(result["garch_after"].summary())




================== DOT_COM_BUBBLE ==================
Significant difference in GARCH parameters? → Yes
LR statistic: 14.1463, p-value: 0.0027

--- GARCH Model (Before Event) ---
                       Zero Mean - GARCH Model Results                        
Dep. Variable:                   None   R-squared:                       0.000
Mean Model:                 Zero Mean   Adj. R-squared:                  0.001
Vol Model:                      GARCH   Log-Likelihood:                2444.27
Distribution:                  Normal   AIC:                          -4876.54
Method:            Maximum Likelihood   BIC:                          -4848.40
                                        No. Observations:                  805
Date:                Mon, Mar 24 2025   Df Residuals:                      805
Time:                        21:22:08   Df Model:                            0
                              Volatility Model                              
                 coef    std err 

In [35]:

def perform_chow_test_from_known_order(full_data, before_data, after_data, p, q):
    try:
        model_full = ARIMA(full_data, order=(p, 0, q)).fit()
        model_before = ARIMA(before_data, order=(p, 0, q)).fit()
        model_after = ARIMA(after_data, order=(p, 0, q)).fit()
        
        SSR_p = np.sum(model_full.resid ** 2)
        SSR_1 = np.sum(model_before.resid ** 2)
        SSR_2 = np.sum(model_after.resid ** 2)
        print(f"[{event}] SSR full: {SSR_p:.4f}, before: {SSR_1:.4f}, after: {SSR_2:.4f}")
        print(f"Lengths → full: {len(full_data)}, before: {len(before_data)}, after: {len(after_data)}\n")


        n1 = len(before_data)
        n2 = len(after_data)
        k = p + q + 1  # intercept + AR + MA

        numerator = max(0, (SSR_p - (SSR_1 + SSR_2))) / k
        denominator = (SSR_1 + SSR_2) / (n1 + n2 - 2 * k)
        F_stat = numerator / denominator
        p_value = 1 - f.cdf(F_stat, k, (n1 + n2 - 2 * k))

        return {
            "F_stat": F_stat,
            "p_value": p_value,
            "significant": "Yes" if p_value < 0.05 else "No",
            "model_order": (p, q)
        }

    except Exception as e:
        return {"error": str(e)}


chow_results = []

for period in structured_periods:
    event = period["event"]
    full_data = period["returns"]
    before_data = full_data.loc[:period["event_date"]]
    after_data = full_data.loc[period["event_date"]:]

    # Get optimal (p,q) for this event
    arma_row = arch_lm_df[arch_lm_df['event'] == event].iloc[0]
    p, q = int(arma_row['p']), int(arma_row['q'])

    result = perform_chow_test_from_known_order(full_data, before_data, after_data, p, q)
    result["event"] = event
    chow_results.append(result)

# Display
chow_df = pd.DataFrame(chow_results)
print(chow_df[["event", "model_order", "F_stat", "p_value", "significant"]])


[dot_com_bubble] SSR full: 0.2696, before: 0.1172, after: 0.1517
Lengths → full: 1509, before: 805, after: 705

[financial_crisis] SSR full: 0.3013, before: 0.0692, after: 0.2337
Lengths → full: 1259, before: 680, after: 580

[flash_crash] SSR full: 0.3095, before: 0.2250, after: 0.0827
Lengths → full: 1009, before: 505, after: 505

[covid_crash] SSR full: 0.1584, before: 0.0591, after: 0.1129
Lengths → full: 1009, before: 504, after: 506

[russia_invasion] SSR full: 0.2021, before: 0.1237, after: 0.0715
Lengths → full: 1008, before: 507, after: 502

              event model_order    F_stat       p_value significant
0    dot_com_bubble      (0, 0)  4.281926  3.868969e-02         Yes
1  financial_crisis      (3, 0)  0.000000  1.000000e+00          No
2       flash_crash      (2, 0)  2.000858  1.122111e-01          No
3       covid_crash      (3, 2)  0.000000  1.000000e+00          No
4   russia_invasion      (0, 3)  8.839006  5.153350e-07         Yes


LR test ARIMA

In [33]:
import numpy as np
from statsmodels.tsa.arima.model import ARIMA
from scipy.stats import chi2

def lr_test_arima(full_data, before_data, after_data, p, q):
    """
    Likelihood Ratio (LR) test for a structural break in ARIMA(p,0,q) parameters.
    - Null (H0): One ARIMA(p,q) model fits the entire sample (restricted).
    - Alt  (H1): ARIMA(p,q) fits separately on before + after, so they can differ (unrestricted).
    
    LR = -2[ logL(full) - (logL(before) + logL(after)) ] ~ Chi-square, df = (p + q + 1)
    """
    try:
        model_full = ARIMA(full_data, order=(p, 0, q)).fit()
        ll_full = model_full.llf

        model_before = ARIMA(before_data, order=(p, 0, q)).fit()
        model_after = ARIMA(after_data, order=(p, 0, q)).fit()
        ll_unrestricted = model_before.llf + model_after.llf

        k = p + q + 1

        LR_stat = -2 * (ll_full - ll_unrestricted)
        p_value = 1 - chi2.cdf(LR_stat, df=k)
        return {
            "LR_stat": LR_stat,
            "p_value": p_value
        }
    except Exception as e:
        return {"error": str(e)}

lr_arima_results = []
for period in structured_periods:
    event_name = period["event"]
    full_data = period["returns"]
    
    before_data = full_data.loc[period["start"]:period["event_date"]].iloc[:-1]
    after_data = full_data.loc[period["event_date"]:]
    if len(before_data) < 5 or len(after_data) < 5:
        lr_arima_results.append({
            "event": event_name,
            "error": "Too few data in sub-samples"
        })
        continue

    row = arch_lm_df.loc[arch_lm_df["event"] == event_name]
    if row.empty or pd.isnull(row.iloc[0]["p"]) or pd.isnull(row.iloc[0]["q"]):
        lr_arima_results.append({"event": event_name, "error": "No ARIMA order found"})
        continue

    p_star = int(row.iloc[0]["p"])
    q_star = int(row.iloc[0]["q"])

    test_out = lr_test_arima(full_data, before_data, after_data, p_star, q_star)
    test_out["event"] = event_name
    test_out["model_order"] = (p_star, q_star)

    if "error" not in test_out:
        sig = "Yes" if test_out["p_value"] < 0.05 else "No"
        test_out["significant"] = sig

    lr_arima_results.append(test_out)

lr_arima_df = pd.DataFrame(lr_arima_results)
print("=== LR test for ARIMA structural break ===")
print(lr_arima_df)


=== LR test for ARIMA structural break ===
      LR_stat       p_value             event model_order significant
0   32.998293  9.223981e-09    dot_com_bubble      (0, 0)         Yes
1  299.833219  0.000000e+00  financial_crisis      (3, 0)         Yes
2  131.071602  0.000000e+00       flash_crash      (2, 0)         Yes
3  -16.574044  1.000000e+00       covid_crash      (3, 2)          No
4   72.205350  7.771561e-15   russia_invasion      (0, 3)         Yes
